## Quantization in Large Language Models

### **Introduction to Quantization**

Quantization is a process used to reduce the memory requirements and computational complexity of large machine learning models. By representing model parameters with lower-precision values, quantization makes it possible to run models more efficiently on devices with limited memory and computational resources.

For large language models (LLMs), quantization can:
- **Reduce Memory Usage:** Lower-precision data types (such as int8) use less memory than higher-precision types (like float32), allowing models to fit into memory-constrained environments.
- **Improve Inference Speed:** By using simpler operations on smaller data types, quantization can reduce the time it takes for a model to process inputs and generate outputs.
- **Preserve Accuracy:** Quantization is carefully designed to minimize the impact on model accuracy, though a trade-off often exists between precision and efficiency.

We will focus on Post-Training Quantization (PTQ), a quantization technique that applies quantization to a pre-trained model. PTQ is a popular method for quantizing large language models because it can be applied to a wide range of models that we may want to use in inference mode. By contrast, quantization-aware training (QAT) requires retraining the model with quantization in mind, which can be more complex and time-consuming.

We will first get a general understanding of quantization by manually implementing two commonly adopted approaches: absmax and minmax (or zero-point). 

Next, we will explore two different ways (one using PyTorch, the other using HuggingFace) to run a **dynamic quantization** (i.e., PTQ where only the weights are quantized, and not the activations). 

## 1. Absmax and Minmax Quantization

The goal of quantization is, remember, mapping continuos values (e.g., float32) into a discrete set of values (e.g., int8). 

So let's create a matrix $W$ and a vector $x$ to be quantized. Let's initialize them randomly (but, just to see what happens, let's set W[0,0] and x[0] = 0).

In [1]:
import torch

torch.random.manual_seed(0)

n_rows = 3
n_cols = 5

W = torch.randn(n_rows, n_cols)
x = torch.randn(n_cols)

W[0,0] = 0
x[0] = 0

Let's first compute the matrix multiplication to observe the result. This is the operation that we typically want to execute, and that we want to quantize.

We will quantize $W$ and $x$ separately, and then multiply them together. Finally, we will need to dequantize the result to compare it with the original result.

In [2]:
out = W @ x
print(out)

tensor([0.4829, 1.3345, 0.2121])


## 1.1 Absmax quantization

In absmax quantization, we use a symmetric range around 0. This means that we need to identify the maximum absolute value in the matrix $W$ and the vector $x$.

We define a function `absmax_quantize` that takes as input any tensor and produces a version of the same tensor, but quantized. 

In [3]:
def absmax_quantize(W):
    # NOTE: we assume that we always map to 8-bit integers

    max_value = W.abs().max()
    scale = 127/max_value

    W_q = (W*scale).round().to(torch.int8)
    return W_q, scale

Notice, we return both the quantized tensor and the scale factor. The scale factor is used to dequantize the tensor. So we might as well define a dequantize function:

In [4]:
def absmax_dequantize(W_q, scale):
    dequantized_W = W_q.float()/scale
    return dequantized_W

Let's get the quantized version of W, and of x. Then, we can check how much we are losing by quantizing the values.

In [5]:
W_q, scale_W = absmax_quantize(W)
x_q, scale_x = absmax_quantize(x)

In [6]:
print(W_q)
print(W)

W_deq = absmax_dequantize(W_q, scale_W)
print(W_deq)
W_diff = W-W_deq
print(W_diff.abs().mean())

tensor([[   0,  -17, -127,   33,  -63],
        [ -82,   24,   49,  -42,  -24],
        [ -35,   11,  -50,   64,  -62]], dtype=torch.int8)
tensor([[ 0.0000, -0.2934, -2.1788,  0.5684, -1.0845],
        [-1.3986,  0.4033,  0.8380, -0.7193, -0.4033],
        [-0.5966,  0.1820, -0.8567,  1.1006, -1.0712]])
tensor([[ 0.0000, -0.2916, -2.1788,  0.5661, -1.0808],
        [-1.4068,  0.4117,  0.8406, -0.7205, -0.4117],
        [-0.6005,  0.1887, -0.8578,  1.0980, -1.0637]])
tensor(0.0039)


In [7]:
print(x_q)
print(x)

x_deq = absmax_dequantize(x_q, scale_x)
print(x_deq)
x_diff = x-x_deq
print(x_diff.abs().mean())

tensor([   0,  -48,   31,  -75, -127], dtype=torch.int8)
tensor([ 0.0000, -0.5663,  0.3731, -0.8920, -1.5091])
tensor([ 0.0000, -0.5704,  0.3684, -0.8912, -1.5091])
tensor(0.0019)


Notice that, in both cases, absmax maps the value 0 to 0. This is a good property, as it allows us to represent the zero value without losing any information. This property stems from the symmetry around 0 we imposed.

However, do note that we are also "wasting" some bits of the range! Can you spot where?

Let's now compute the matrix multiplication between W_q and x_q. 

In [8]:
product = W_q @ x_q
print(product)

tensor([101, -91, -28], dtype=torch.int8)


Can you see that there's something wrong? Let's see what one of the rows of W_q and x_q contain:

In [9]:
W_q[0], x_q

(tensor([   0,  -17, -127,   33,  -63], dtype=torch.int8),
 tensor([   0,  -48,   31,  -75, -127], dtype=torch.int8))

The dot product of these two vectors definitely isn't what we get as the first number of the matrix multiplication -- i.e. (W_q @ x_q)[0]. Indeed, we can run as int16, and see that the result is quite different:

In [10]:
out_q = W_q.to(torch.int16) @ x_q.to(torch.int16)
print(out_q)

tensor([2405, 6565,  996], dtype=torch.int16)


The result of the dot product overflows the int8 range. This is a well-known problem. Indeed, the accumulation of results, in quantization, is typically done with higher precision than the single values. This is tricky to do in pure Python/PyTorch, but can be done efficiently in other ways.

Let's stick to the simple approach for now. 

To get the correct result, we need to dequantize the result. This is done by multiplying the result by the scale factor of the two operands.

In [11]:
out_deq = out_q / (scale_W*scale_x)

print(out_deq)
print(out)

tensor([0.4903, 1.3383, 0.2030])
tensor([0.4829, 1.3345, 0.2121])


Remember, our goal was `out`. How much did we lose by quantizing and dequantizing?

In [13]:
out_diff = (out - out_deq).abs().mean()
out_diff

tensor(0.0068)

## 2. Minmax Quantization

In minmax quantization, we use the minimum and maximum values in the matrix $W$ and the vector $x$ to define the range. In this way, we get a range that is as tight as possible around the values we are quantizing. This will, however, change the zero value, which will not be mapped to 0 anymore.



In [43]:
def minmax_quantize(W):

    delta = W.max() - W.min()
    scale = 255 / delta 
    zero_point = -(128*W.max() + 127*W.min()) / delta
    W_q = (W * scale + zero_point).round().to(torch.int8)
    return W_q, scale, zero_point

def minmax_dequantize(W_q, scale, zero_point):
    return (W_q.float() - zero_point) / scale

In [44]:
W_q, scale_W, zero_point_W = minmax_quantize(W)
x_q, scale_x, zero_point_x = minmax_quantize(x)

Let's see the results for $W$ (same considerations will apply for $x$).

In [45]:
print(W_q)
print(W)

W_deq = minmax_dequantize(W_q, scale_W, zero_point_W)
print(W_deq)
W_diff = (W - W_deq).abs().mean()
print(W_diff)
print("zero point", zero_point_W)

tensor([[  41,   19, -128,   86,  -43],
        [ -67,   73,  107,  -15,   10],
        [  -5,   56,  -25,  127,  -42]], dtype=torch.int8)
tensor([[ 0.0000, -0.2934, -2.1788,  0.5684, -1.0845],
        [-1.3986,  0.4033,  0.8380, -0.7193, -0.4033],
        [-0.5966,  0.1820, -0.8567,  1.1006, -1.0712]])
tensor([[-0.0054, -0.2883, -2.1788,  0.5733, -1.0857],
        [-1.3943,  0.4061,  0.8434, -0.7256, -0.4041],
        [-0.5970,  0.1875, -0.8542,  1.1006, -1.0728]])
tensor(0.0031)
zero point tensor(41.4189)


First, notice that 0 no longer maps to 0! Indeed, it maps to zero_point_W (after rounding). This implies that the dequantization of 0 will no longer be 0. This may be a problem!

But, notice that we are using the full range of the int8 values. This means that we are not wasting any bits of the range! (the minimum value is -128, the maximum value is 127). This can also be seen in the average absolute error, which is lower than what we had with absmax.

Similarly to what we did before, let's compute the output of the operation, and then dequantize it!

In [46]:
out_q = W_q.to(torch.int16) @ x_q.to(torch.int16)
out_q

tensor([-11420,   7877,  -3767], dtype=torch.int16)

The dequantification is a bit trickier, in this case. Can you figure out why we need the following operations?

Hint: consider the transformation we are applying to each value (value * scale + zero_point). What happens when we compute the dot product?

In [47]:
out_deq = (out_q - W.shape[1] * zero_point_W * zero_point_x - W.sum(axis=1) * scale_W * zero_point_x - x.sum() * scale_x * zero_point_W) / (scale_W * scale_x)

In [48]:
print(out_deq)
print(out)
print((out_deq - out).abs().mean())

tensor([0.4812, 1.3489, 0.2222])
tensor([0.4829, 1.3345, 0.2121])
tensor(0.0087)


<span style="color:red">Extra stuff!</span>

We could have computed scales and zero points at different granularities (e.g., for each row, or column of $W$). How would that have changed the results? What changes would we have to do to the code?

Yes, we could have been computing scale and zero-point fo every column of W (x doesn't change, has only 1 row) and get better results. The code would have been like this:

In [50]:
def minmax_quantize_per_column(W):

    max_vals = W.max(dim=0).values
    min_vals = W.min(dim=0).values
    deltas = max_vals - min_vals
    scales = 255 / deltas
    zero_points = -(128*max_vals + 127*min_vals) / deltas
    W_q = (W * scales + zero_points).round().to(torch.int8)
    return W_q, scales, zero_points

def minmax_dequantize_per_column(W_q, scales, zero_points):
    return (W_q.float() - zero_points) / scales

In [52]:
W_q, scales_W, zero_points_W = minmax_quantize_per_column(W)
x_q, scales_x, zero_points_x = minmax_quantize_per_column(x)

out_q = W_q.to(torch.int16) @ x_q.to(torch.int16)

W_deq = minmax_dequantize_per_column(W_q, scales_W, zero_points_W)
x_deq = minmax_dequantize(x_q, scale_x, zero_point_x)

out_deq = W_deq @ x_deq

print((out_deq - out).abs().mean())

tensor(0.0046)


# Dynamic quantization

In this second part, we will apply dynamic quantization by using PyTorch or HuggingFace (with BitsAndBytes). We will quantize both to 8 and to 4 bits, and we will see how that affects LLMs (in terms of memory and speed). 

In [2]:
import torch
import os
import time 
from transformers import AutoTokenizer, AutoModelForCausalLM

c:\Users\Utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from huggingface_hub import login

# Login to the Hugging Face model hub to be able to upload models
with open("../hf_token.txt", "r") as f:
    token = f.read()
    f.close()

login(token=token)


First, let's load our model (Llama 3.2 1B) and let's see some base statistics (memory usage, inference time).

In [4]:
model_id = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token

In [2]:
def get_model_size(model):
    """Get the size of the model in MB"""
    torch.save(model.state_dict(), "temp.pth")
    size = os.path.getsize("temp.pth") / 1e6  # size in "MB" (technically, it should be 1024**2, but we approximate to 1e6 to get an easier conversion #params <=> MB)
    os.remove("temp.pth")
    return size

print(f"Model size before quantization {(get_model_size(model)):.2f} MB")

NameError: name 'os' is not defined

Wait, wasn't Llama 1B supposed to be 4GB (4 bytes * 1B parameters)? Why do we get ~ 5 GB (i.e., 1.25B parameters)?

We are not considering the parameters used in the embedding layer (you can count how many parameters you have in the embedding layer and see that it matches the difference). 

Additionally, the count does not include the `lm_head`, i.e. the layer used to go from the hidden states to the logits. This is because in Llama (and other models) the `lm_head` is shared with the embedding layer, so it is not counted twice.

In [5]:
text = "The secret of life is"
# Notice we use a batch of 20 sentences -- we will get better results
# on quantized models when processing a batch of inputs
text = [text]*20

inputs = tokenizer(text, return_tensors='pt')

tic = time.time()

with torch.no_grad():
    baseline_output = model.generate(**inputs, max_new_tokens=100)

elapsed_time = time.time() - tic

baseline_decoded = tokenizer.decode(baseline_output[0], skip_special_tokens=True)

print("Baseline model output:", baseline_decoded)
print("\nTime taken for baseline model:", elapsed_time)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Baseline model output: The secret of life is to know something about yourself, to be yourself, and to be a little different from the rest of the world. If you are different, you are right. If you are not different, you are wrong. You have to be yourself to be happy.
The best way to find yourself is to lose yourself in the service of others.
You have to be yourself to be happy.
You have to be yourself to be happy.

Time taken for baseline model: 31.20296883583069


Dynamic quantization applies lower precision to model weights and activations at runtime. This method doesn’t require modifications to the model architecture or retraining, which makes it relatively easy to apply.

- **Advantages:** 
  - Quick to implement with minimal changes. No calibration step is needed.

- **Limitations:** 
  - Activations are not pre-quantized, meaning some precision is maintained but at the cost of slightly higher resource use at inference time.


We use **[TorchAO](https://docs.pytorch.org/ao/stable/api_ref_quantization.html?utm_source=chatgpt.com)** to apply dynamic quantization to a model. TorchAO is the new quantization framework that replaces the deprecated `torch.ao.quantization` APIs.  

We can specify a set of layer types to be quantize. Let's stick with Linear layers. We specify the desired type, and off we go!

In [6]:
import torch
from torchao.quantization import quantize_, Int8DynamicActivationInt8WeightConfig

quantize_(
    model,
    Int8DynamicActivationInt8WeightConfig(),
    filter_fn=lambda m, name: isinstance(m, torch.nn.Linear),
    device="cpu",
)

print(f"Model size after quantization {get_model_size(model):.2f} MB")

c:\Users\Utente\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\Context.cpp:85.)
  _C._set_float32_matmul_precision(precision)


Model size after quantization 2288.82 MB


Okay -- 2.3GB? Why not 5GB / 4 = 1.25GB? After all, we are going from float32 to int8. 

That's correct -- technically. Except, we are only encoding linear layers, and not the embedding layer. That means that, of the original 1.25B parameters, we are only quantizing 1B. The rest, in the embedding layer, is kept as float32.

If you run the numbers, though, you should still find a problem: 1B * 1 byte + 0.25B * 4 bytes = 2GB. What about the rest? There's one more thing: remember, the `lm_head` was shared with the Embedding layer. However, since it is "copied" into a linear layer in Llama, the quantization process will quantize it as well. So that's an extra 0.25B parameters encoded as int8 -- hence 2.3GB.

Finally, we could technically also quantize the embeddings (it has been introduced in later versions of PyTorch), but for simplicity we will not do it here (it would require some additional steps).

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, weight=LinearActivationQuantizedTensor(activation=<function _int8_symm_per_token_reduced_range_quant at 0x0000013CB86062A0>, weight=AffineQuantizedTensor(shape=torch.Size([2048, 2048]), block_size=(1, 2048), device=cpu, _layout=PlainLayout(), tensor_impl_dtype=torch.int8, quant_min=None, quant_max=None)))
          (k_proj): Linear(in_features=2048, out_features=512, weight=LinearActivationQuantizedTensor(activation=<function _int8_symm_per_token_reduced_range_quant at 0x0000013CB86062A0>, weight=AffineQuantizedTensor(shape=torch.Size([512, 2048]), block_size=(1, 2048), device=cpu, _layout=PlainLayout(), tensor_impl_dtype=torch.int8, quant_min=None, quant_max=None)))
          (v_proj): Linear(in_features=2048, out_features=512, weight=Line

In [10]:
text = "The secret of life is"

text = [text]*20
inputs = tokenizer(text, return_tensors='pt')

tic = time.time()

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100)

elapsed_time = time.time() - tic

output_decoded = tokenizer.decode(output[0], skip_special_tokens=True)

print("Quantized model output:", output_decoded)
print("\nTime taken for baseline model:", elapsed_time)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Quantized model output: The secret of life is to live it to the fullest, and to make it so that it seems like a dream.
The secret of life is to live it to the fullest, and to make it so that it seems like a dream.
It's not what you have, or who you are, it's what you do with what you have, or who you are.
It's not what you have, or who you are, it's what you do with what you have, or who you are.
The secret of

Time taken for baseline model: 16.525156497955322


Hugging Face provides several built-in quantization options, each suited to different model and deployment needs:
https://huggingface.co/docs/transformers/v4.46.0/quantization/overview

For this lab, we will use `Quanto`.

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig
import torch
import os
import time 
    
model_id = "meta-llama/Llama-3.2-1B"

# Quantize to 8-bit weights
quant = QuantoConfig(weights="int8")

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant,
    device_map="cpu",      
)

print(f"Model size after quantization: {get_model_size(model)} MB")

Model size after quantization: 2025.673283 MB


In [5]:
model.model.layers[0].self_attn.q_proj.weight

<class 'optimum.quanto.tensor.weights.qbytes.WeightQBytesTensor'>(tensor([[-49,  19,  59,  ..., -19, -24,  40],
        [ 13,  69,  73,  ..., -39, -17,   7],
        [ 17,  13,  34,  ..., -40, -36, -22],
        ...,
        [ 16,  15,  43,  ..., -41, -17, -18],
        [ 36, -48,  54,  ..., -18, -39, -22],
        [-14, -27, -11,  ...,  32,   7,   0]], dtype=torch.int8), scale=tensor([[0.0004],
        [0.0009],
        [0.0011],
        ...,
        [0.0019],
        [0.0007],
        [0.0019]]), dtype=torch.float32)

In [7]:
text = "The secret of life is"
inputs = tokenizer([text]*20, return_tensors="pt")

tic = time.time()

with torch.no_grad():
    baseline_output = model.generate(**inputs, max_new_tokens=100)

elapsed_time = time.time() - tic

output_decoded = tokenizer.decode(baseline_output[0], skip_special_tokens=True)

print("\nquantized model output:", output_decoded)
print("\nTime taken for baseline model:", elapsed_time)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



quantized model output: The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The secret of life is to enjoy every moment of every day.
The

Time taken for baseline model: 67.56964612007141
